# Construindo um vector database em um banco de dados relacional

Nesse notebook iremos inserir um vector database em um banco de dados relacional.

O objetivo é demonstrar que podemos inserir os vetores e fazer queries com eles também.


## 1 - Configuração

In [1]:
# Recarrega automaticamente quando há modificação na pasta src/
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import sqlite3
from pathlib import Path

In [3]:
# Configuração de diretórios
HERE = os.path.abspath(".")
MODULES = Path(HERE).parent
DATA = Path(HERE).parent / "data"

sys.path.insert(
    0, str(MODULES)
)  # para ser possível importar os módulos personalizados da pasta src/

In [4]:
from src.io import SQLiteIO

sqlite = SQLiteIO(DATA / "2_processed" / "database-vector.db")

## 2 - Criando tabela

Diferente do notebook 01, aqui as nossas informações são "compiladas" em uma única coluna que possui o nosso vetor.

Nesse caso, precisamos que a nossa coluna de vetores seja um objeto binário. Sendo assim, o nosso schema fica:

* Id: integer -> Chave primária;
* Vector: blob


In [5]:
sqlite.query(
    """
        CREATE TABLE IF NOT EXISTS vectors (
            id INTEGER PRIMARY KEY,
            vector BLOB NOT NULL
        )
    """
)

## 3 - Criando vetores aleatórios

Para esse notebook, simularemos os vetores já construídos a partir das nossas informações. Porém em casos mais reais, utilizaríamos um modelo de embedding que seria responsável por transformar nossa informação nos vetores.

O [Hugging Face](https://huggingface.co/) possui diversos modelos Open Source que se propõem a construir os embeddings por nós, porém é possível criar um modelo personalizado a partir dos seus conhecimentos.

*O Hugging Face é um hub de conteúdos focados em inteligência artificial de colaboração livre, possuindo modelos, datasets etc..*


In [6]:
from src.utils import create_random_vector

vec1 = create_random_vector((0, 10), 4)
vec2 = create_random_vector((0, 10), 4)
print(f"Vector 1: {vec1} \nVector 2: {vec2}")

Vector 1: [6.8754487  2.17723056 9.84518115 2.99416775] 
Vector 2: [2.62309813 7.2275818  2.50797115 5.42794579]


## 4 - Populando tabela com os vetores

Para essa tarefa precisamos transformar o nosso numpy array para um bytes.

In [7]:
from src.utils import transform_np_array

sqlite.query(
    "INSERT INTO vectors (vector) VALUES(?)",
    (sqlite3.Binary(transform_np_array(vec1, "bytes")),),
)
sqlite.query(
    "INSERT INTO vectors (vector) VALUES(?)",
    (sqlite3.Binary(transform_np_array(vec2, "bytes")),),
)

sqlite.close()

E nesse caso, nossos vetores são armazenados como objetos binários.

In [8]:
sqlite.read(table_name="vectors")

,id,vector
0,1,b'h\xd3\x08\xa0u\x80\x1b@\xed\x91\xae\xdb\xf7j...
1,2,b'j\xfcy\xdf\x1a\xfc\x04@xn\x034\x0b\xe9\x1c@\...


## 5 - Buscando o vetor mais próximo da query

Da mesma forma que podemos filtrar as nossas queries no banco de dados estruturado padrão, nós conseguimos fazer com os nossos vetores que é chamado de similaridade. A similaridade é um cálculo feito a partir do nosso "vetor query" - que é o vetor com o que perguntamos - comparado aos vetores armazenados no nosso banco de dados.

A similaridade é - de maneira "grosseira" - cálculo da distância entre pontos de dimensão n. Possuem algumas métricas "clássicas" que podemos utilizar como:

* Distância euclidiana
* Cosseno
* Manhattan

Porém podemos construir outras métricas para calcular essa similaridade. No nosso notebook utilizaremos uma métrica simples que é a diferença absoluta entre os dois vetores.


In [10]:
vector_query = create_random_vector((0, 10), 4)
print(vector_query)
vector_query = transform_np_array(vector_query, "bytes")


[2.66984295 5.39511837 7.21057865 8.73615597]


Na query abaixo utilizaremos a métrica de diferença absoluta entre os vetores e ordenaremos em ordem ascendente.

In [11]:

result = sqlite.query(
    "SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC",
    (sqlite3.Binary(vector_query),),
)
row = result.fetchone() # Busca o primeiro resultado
print(row)

(b'h\xd3\x08\xa0u\x80\x1b@\xed\x91\xae\xdb\xf7j\x01@\xd5{W\x95\xbb\xb0#@\xda7+8\x0e\xf4\x07@',)


Desserializando o nosso vetor que está em bytes, conseguimos ver o nosso vetor na sua forma original.

In [12]:
from src.utils import desserialize_bytes_array

print(desserialize_bytes_array(row[0]))

[6.8754487  2.17723056 9.84518115 2.99416775]
